In [1]:
from selenium import webdriver
import time
import csv
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
driver=webdriver.Chrome()

In [2]:
#Creating a Template URL
def get_url(search_term):
    """ Generate a URL for the search term"""
    template="https://www.amazon.com/s?k={}&crid=3JKZD4EXHPZC&sprefix=%2Caps%2C432&ref=nb_sb_ss_recent_1_0_recent"
    search_term=search_term.replace(' ','+')
    return template.format(search_term)

In [3]:
url=get_url('keyboard')
print(url)

https://www.amazon.com/s?k=keyboard&crid=3JKZD4EXHPZC&sprefix=%2Caps%2C432&ref=nb_sb_ss_recent_1_0_recent


In [4]:
driver.get(url)

In [5]:
# Reading The HTML websit
soup=BeautifulSoup(driver.page_source,'html.parser')
results=soup.find_all('div',{'data-component-type':'s-search-result'})
len(results)

22

In [6]:
# Extracting the description
item=results[0]
item.h2.a.text.strip()

'Amazon Basics Low-Profile Wired USB Keyboard with US Layout (QWERTY), Matte Black'

In [7]:
# Price of the item
price_parent=item.find('span','a-price')
price=price_parent.find('span','a-price-whole').text.strip()
price

'17.'

In [8]:
def get_url(search_term):
    """ Generate a URL for the search term"""
    template="https://www.amazon.com/s?k={}&crid=3JKZD4EXHPZC&sprefix=%2Caps%2C432&ref=nb_sb_ss_recent_1_0_recent"
    search_term=search_term.replace(' ','+')

    #add term query to url
    url=template.format(search_term)

    #add page query place holder
    url += '&page{}'

    return url

In [9]:
def extract_record(item):
    # Extract description and URL
    atag = item.h2.a
    description = atag.text.strip()  # Get the text of the 'a' tag (product description)
    url = "https://www.amazon.com" + atag.get('href')  # Construct the complete URL for the product

    try:
        # Extract price
        price_parent = item.find('span', 'a-price')  # Find the parent element containing the price
        price = price_parent.find('span', 'a-price-whole').text.strip()  # Get the price text
    except AttributeError:
        # If price information is not available, return None
        return None

    try:
        # Extract rating and review count
        rating = item.i.text  # Get the rating text
        review_count = item.find('span', {'class': 'a-size-base'}).text  # Get the review count text
    except AttributeError:
        # If rating or review count information is not available, set them to empty strings
        rating = ''
        review_count = ''

    # Create a tuple containing the extracted data
    result = (description, price, rating, review_count, url)

    return result


In [10]:

def search_query(search_term):
    """Run main program routine"""

    # Start up the webdriver
    # This code is for google chrome driver if running else where then change this Chrome driver path with your own path,
    chrome_options = Options()
    driver = webdriver.Chrome(options=chrome_options)

    records = []  # Create an empty list to store the extracted data
    url = get_url(search_term)  # Get the initial URL for the search term

    # Loop through 20 pages of search results
    for page in range(1, 21):
        driver.get(url.format(page))  # Load the page in the webdriver
        soup = BeautifulSoup(driver.page_source, 'html.parser')  # Parse the HTML content of the page
        results = soup.find_all('div', {'data-component-type': 's-search-result'})  # Find all search result items

        # Loop through each search result item and extract data
        for item in results:
            record = extract_record(item)  # Extract data from the current search result item
            if record:
                records.append(extract_record(item))  # Append the extracted data to the records list

    driver.close()  # Close the webdriver

    df = pd.DataFrame(records, columns=['Description', 'Price(Rs)', 'Rating', 'Review Count', 'Url of the Product'])

    # Display the DataFrame as a table in the console
    display(df)

    filename = search_term + ".csv"  # Create a CSV file name based on the search term

    # Save the data to a CSV file
    with open(filename, "w", newline='', encoding='utf-8') as f:
        writer = csv.writer(f)  # Create a CSV writer
        writer.writerow(['Description', 'Price(Rs)', 'Rating', 'Review Count', 'Url of the Product'])  # Write header row
        writer.writerows(records)  # Write the extracted records to the CSV file

In [11]:
search_query('Keyboard')

,Description,Price(Rs),Rating,Review Count,Url of the Product
0,Amazon Basics Low-Profile Wired USB Keyboard w...,17.,4.5 out of 5 stars,4.5,https://www.amazon.com/sspa/click?ie=UTF8&spc=...
1,"Wireless Keyboard and Mouse Combo, cimetech Co...",33.,4.3 out of 5 stars,4.3,https://www.amazon.com/sspa/click?ie=UTF8&spc=...
2,"Dacoity Gaming Keyboard, 104 Keys All-Metal Pa...",26.,4.5 out of 5 stars,Amazon's Choice: Overall Pick,https://www.amazon.com/All-Metal-Dacoity-Multi...
3,"Dilter Wired Keyboard, Full-Sized Typewriter K...",29.,4.5 out of 5 stars,4.5,https://www.amazon.com/Dilter-Full-Sized-Typew...
4,"NPET K10 Wired Gaming Keyboard, LED Backlit, S...",19.,4.5 out of 5 stars,4.5,https://www.amazon.com/NPET-Floating-Keyboard-...
...,...,...,...,...,...
435,Corsair K55 PRO LITE RGB Wired Membrane Gaming...,29.,4.6 out of 5 stars,4.6,https://www.amazon.com/Corsair-Backlighting-In...
436,"Lovaky MK98 Wireless Keyboard, 2.4G Ergonomic ...",20.,4.3 out of 5 stars,4.3,https://www.amazon.com/Lovaky-Wireless-Ergonom...
437,"NPET K10 Wired Gaming Keyboard, RGB Backlit, S...",21.,4.7 out of 5 stars,4.7,https://www.amazon.com/NPET-K10-Water-Resistan...
438,"Wireless Keyboard and Mouse Combo, MARVO 2.4G ...",19.,4.3 out of 5 stars,4.3,https://www.amazon.com/Wireless-Keyboard-MARVO...
